In [ ]:
import asyncio
import socketio
sio = socketio.AsyncClient()

file1 = "ru_lock_unlock_normal1_simple.log"
file2 = "ru_lock_unlock_dpd_hw_fault_simple.log"

await sio.connect('http://127.0.0.1:8000', namespaces=['/TextAnalysis/FileContainer', '/TextAnalysis/TextFileCompare'])
await sio.emit('new_file', [f'D:\\Projects\\ericsson_flow\\new_files\\{file1}'], namespace='/TextAnalysis/FileContainer')
await asyncio.sleep(0.2)
await sio.emit('load_config', 'D:\\projects\\ericsson_flow\\new_files\\6419config.ecfg', namespace='/TextAnalysis/FileContainer')
await asyncio.sleep(10)
await sio.emit('new_file', [f'D:\\Projects\\ericsson_flow\\new_files\\{file2}'], namespace='/TextAnalysis/FileContainer')
await asyncio.sleep(0.2)
await sio.emit('load_config', 'D:\\projects\\ericsson_flow\\new_files\\6419config.ecfg', namespace='/TextAnalysis/FileContainer')
await asyncio.sleep(10)
await sio.emit('exec', {'first_file_namespace': f'/TextAnalysis/FileContainer/{file1}', 'second_file_namespace': f'/TextAnalysis/FileContainer/{file2}'}, namespace='/TextAnalysis/TextFileCompare')

In [ ]:
sio1 = socketio.AsyncClient()
await sio1.connect('http://127.0.0.1:8000', namespaces=[
'/TextAnalysis/FileContainer',
'/TextAnalysis/TextFileCompare',
'/TextAnalysis/FileContainer/ru_lock_unlock_normal1_simple.log/TextFileFunction/SearchFunction/txlProcBranch0',
'/TextAnalysis/FileContainer/ru_lock_unlock_dpd_hw_fault_simple.log/TextFileFunction/SearchFunction/txlProcBranch0'
])

In [ ]:
model1 = {}
model2 = {}

def set_data1(model):
    model1.update(model)
def set_data2(model):
    model2.update(model)
await sio1.emit('model', namespace='/TextAnalysis/FileContainer/ru_lock_unlock_normal1_simple.log/TextFileFunction/SearchFunction/txlProcBranch0', callback=set_data1)
await asyncio.sleep(0.2)
await sio1.emit('model', namespace='/TextAnalysis/FileContainer/ru_lock_unlock_dpd_hw_fault_simple.log/TextFileFunction/SearchFunction/txlProcBranch0', callback=set_data2)

In [ ]:
import re
string = '       air_mongoose com_ericsson_trithread INFO cpu_id  process txlProcBranch fileAndLine spiMaster cc  msg Warning Slow response to SPI_SEND_REQ for device paCtrlDevice   took ms '
re.sub(' '+"+", ' ', string)

In [ ]:
s = 'Power measurement'
for item in model1['res_clean_lines']:
    if s in item:
        print('model1', item)
        
for item in model2['res_clean_lines']:
    if s in item:
        print('model2', item)

In [ ]:
	[14:56:47.927884604] (+0.001409563) air6419_mongoose com_ericsson_trithread:INFO: { cpu_id = 2 }, { process = "WorkerTaskAas", fileAndLine = "algAasHelper.cc:659", msg = "Missing subbandMappingExt param, using default fullband instead." }

In [ ]:
model2['res_clean_lines']

In [ ]:
class A(object):
    def __init__(self):
        self.a = 1
        self.b = 2
        
    def __getattribute__(self, obj):
        print(obj)
        return object.__getattribute__(self, obj)
    
c = A()
c.a * c.b

In [ ]:
# from asyncio import get_event_loop
from text_analysis import *

# loop = get_event_loop()
# loop.run_until_complete(TextAnalysisModel('parallel'))

web.run_app(app, host="127.0.0.1", port=8000)

In [ ]:
from dateutil.parser import parse as dp

str(dp('221210-12:40:51', yearfirst=True))

In [ ]:
if '2022-12-11 01:12:21.105110' > '2022-12-11 01:12:22':
    print(True)

In [ ]:
import re
import pandas as pd
from parse import parse

str1 = '230228-16:19:06+0100 172.21.11.66 23.0b MSRBS_NODE_MODEL_22.Q2_566.28125.116_3317 stopfile=/tmp/11737'
str2 = 'BXP_7:     769,dl-1  disabled        false    essFdd150_Id2       B     dl    2162500         BandI      0,       0    469            469     SETUP     SETUP'
# exp_extract = '{}MSRBS_NODE_MODEL_{version}_{}'
exp_extract = '{bxp}:{:s}{carrierId}{:s}{Enabled}{:s}{RealRelease}{:s}{carrierType}{:s}{rfPort}{:s}dl{:s}{Frequency}{:s}{Band}{:s}{Arfcn_min}{:s}{Arfcn_max}{:s}{Power}{:s}{ReservedPower}{:s}{tr}{:s}{bdconf}'
r_extract = parse(exp_extract, str2)
print(r_extract.named)
# print(r_extract)

In [ ]:
lines = []
for index in self.text_file.data['test'].res_lines:
    lines.append(self.text_file.lines[index])

with open('D:\\projects\\ericsson_flow\\new_files\\ru_lock_unlock_normal2_simple.log', 'w', encoding='utf-8') as f:
    f.write(''.join(lines))
    
# jesd|dfe|radiosw|rProxyMedian.cc # (txlProcBranch0|TxBranchCtrl0).*event com_ericsson_trithread:INFO

In [ ]:
data = self.text_file.data['dl'].res_key_value
file = pd.DataFrame(pd.Series(data['Band']['value']), columns=['Band'])

file['rfPort'] = pd.Series(data['rfPort']['value'])
file['rfPort'] = 'Port_' + file['rfPort']
file['carrierType'] = pd.Series(data['carrierType']['value'])
file['Power'] = pd.Series(data['Power']['value'])

result = {
            'SiteName': self.text_file.file_name.split('_radio4480')[0],
            'Version': self.text_file.data['version'].res_key_value['version']['value'][0],
            'RadioQuantity': len(self.text_file.data['device'].res_lines)
         }
for band,port,carrier,power in file.values:
    result[band+'_'+port+'_carrier_config'] = carrier
    result[band+'_'+port+'_output_power_config'] = power

self.result = result

In [ ]:
import json
import pandas as pd

data = pd.read_csv('C:\\Users\\MSI-NB\\Downloads\\6651_config_Denmark_68_Nodes\\KaiHong_4480config_origin_data.csv')
data = data.fillna('')
columns = ['SiteName', 'Bxp', 'Version']
tmpl = sorted(list(set(data.columns).difference(set(columns))))
band1 = []
band3 = []
for c in tmpl:
    if 'BandIII_' in c:
        band3.append(c)
    elif 'BandI_' in c:
        band1.append(c)
        
band1.extend(band3)
columns = band1
total = len(data)
res = data.groupby(columns).size().reset_index(name='RadioAmount')
for index,item in enumerate(json.loads(res.to_json(orient='records'))):
    band1_dbm = []
    band3_dbm = []
    total_dbm = 0
    for key in item.keys():
        if ('output_power_BandIII' in key) & (item[key] != ''):
            band3_dbm.append(item[key])
        elif ('output_power_BandI' in key) & (item[key] != ''):
            band1_dbm.append(item[key])
    band1_dbm = sum(band1_dbm) if len(band1_dbm) > 0 else 0
    band3_dbm = sum(band3_dbm) if len(band3_dbm) > 0 else 0
    total_dbm = band1_dbm + band3_dbm
    res.loc[index, ['BandI_dBm', 'BandIII_dBm', 'BandI+BandIII_dBm']] = [band1_dbm,band3_dbm,total_dbm]
res['Percentage'] = res['RadioAmount'] / total
res.to_csv('C:\\Users\\MSI-NB\\Downloads\\6651_config_Denmark_68_Nodes\\6651_config_Sweden_204_Nodes_config_summary.csv', index=False)

In [ ]:
items = self.text_analysis.batch_statistic.table['table'].values
data = pd.DataFrame()
for index, item in enumerate(items):
    print(index)
    try:
        tmp = pd.DataFrame(item)
    except Exception as e:
        print(str(e))
        continue
    data = data.append(tmp, ignore_index=True)
    
columns = ['SiteName', 'Bxp', 'Version']
tmpl = sorted(list(set(data.columns).difference(set(columns))))
band1 = []
band3 = []
for c in tmpl:
    if 'BandIII_' in c:
        band3.append(c)
    elif 'BandI_' in c:
        band1.append(c)
        
columns.extend(band1)
columns.extend(band3)
data = data.reindex(columns=columns)
data.to_csv('C:\\Users\\MSI-NB\\Downloads\\6651_config_Denmark_68_Nodes\\KaiHong_4480config_origin_data.csv', index=False)

In [ ]:
data = self.text_file.data['dl'].res_key_value
file = pd.DataFrame(pd.Series(data['Band']['value']), columns=['Band'])

file['bxp'] = pd.Series(data['bxp']['value'])
file['rfPort'] = pd.Series(data['rfPort']['value'])
file['rfPort'] = 'Port_' + file['rfPort']
file['carrierType'] = pd.Series(data['carrierType']['value'])
file['Power'] = pd.Series(data['Power']['value'])

result = []
for bxp in set(file['bxp'].values):
    item = {
                'SiteName': self.text_file.file_name.split('.log')[0],
                'Bxp': bxp,
                'Version': self.text_file.data['version'].res_key_value['version']['value'][0]
             }
    tmp = file.loc[(file['bxp'] == bxp), :].reset_index(drop=True)
    for band,bxp,port,carrier,power in tmp.values:
        if band in ['BandI', 'BandIII']:
            cc = band+'_'+port+'_carrier_config'
            pc = band+'_'+port+'_output_power_config'
            dBm = 'output_power_' + band+'_'+port + '_dBm'
            if cc in item:
                item[cc] = carrier + ' + ' + item[cc] if carrier < item[cc] else item[cc] + ' + ' + carrier
                item[pc] = power + ' + ' + item[pc] if carrier < item[cc] else item[pc] + ' + ' + power
                item[dBm] = item[dBm] + round(10**(int(power)/100)/1000, 1)
            else:
                item[cc] = carrier
                item[pc] = power
                item[dBm] = round(10**(int(power)/100)/1000, 1)
    if len(item.keys())> 3:
        result.append(item)

self.result = result

In [ ]:
file_path = 'D:\\projects\\ericsson_flow\\CC summary V2,0.xlsx'
output_path = 'D:\\projects\\ericsson_flow\\output'

sheet_names = pd.read_excel(file_path, sheet_name=None).keys()
sheet_names

In [ ]:
pd.read_excel(file_path, sheet_name='Claro_4480_cc_SW', engine='openpyxl')

In [1]:
import pandas as pd

file_path = 'D:\\projects\\ericsson_flow\\CC summary V2,0.xlsx'
output_path = 'D:\\projects\\ericsson_flow\\output'

print('Loading file...')
sheet_names = pd.read_excel(file_path, sheet_name=None).keys()
for sheet_name in sheet_names:
    print(f'Start Handle {sheet_name}...')
    data = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
    result = []
    x= list(data['SerialNo'].values)
    sns = sorted(set(x), key=x.index)
    for sn in sns:
        radio = data.loc[(data['SerialNo'] == sn)&(data['MaxTxPower_br'].notna()), :].reset_index(drop=True)
        if len(radio) == 0:
            continue
        item = {'SerialNo': sn, 'SwPack': list(radio['SwPack'].values)[0]}
        for port,band,power in radio[['rfPortId', 'freqBand', 'MaxTxPower_br']].values:
            if band in [1,3]:
                item['Band'+str(int(band))+'_Port'+port+'_MaxTxPower_br'] = power

        band1_power = []
        band3_power = []
        for key in item.keys():
            if 'Band1' in key:
                band1_power.append(item[key])
            if 'Band3' in key:
                band3_power.append(item[key])

        item['Band1_MaxTxPower_br'] = sum(band1_power) if len(band1_power) > 0 else 0
        item['Band3_MaxTxPower_br'] = sum(band3_power) if len(band3_power) > 0 else 0
        item['Band1+Band3_MaxTxPower_br'] = item['Band1_MaxTxPower_br'] + item['Band3_MaxTxPower_br']
        result.append(item)
    data = ''
    del data
    origin_data = pd.DataFrame(result)
    origin_data.to_csv(f'{output_path}\\{sheet_name}_Origin_Data.csv', index=False)

    origin_data = origin_data.fillna('')
    total = len(origin_data)
    summary = origin_data.groupby(list(origin_data.columns[2:])).size().reset_index(name='RadioAmount')
    summary['Percentage'] = summary['RadioAmount'] / total
    summary.to_csv(f'{output_path}\\{sheet_name}_Summary.csv', index=False)

Loading file...
Start Handle Claro_4480_cc_SW...
Start Handle Vivo_4480_cc_SW...


In [ ]:
import os

def iterate_files_in_directory(directory):
    # iterate over files in
    # that directory
    res = []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            res.append(filename)
    return res

path = 'D:\\projects\\ericsson_flow\\ErrorLog'
result = []
for file in iterate_files_in_directory(path):
    with open(path+'\\'+file, 'r') as f:
        lines = f.readlines()
        tmp = []
        for line in lines:
            tmp.append(file.replace(' ','_').replace('.txt','')+':'+line)
    result.extend(tmp)
with open(path+'\\'+'DotErrorCase.log', 'w', encoding='utf-8') as f:
    f.write(''.join(result))

In [ ]:
class MyClass:
    def __init__(self):
        self.my_variable = 0
    
    def my_method(self):
        pass
    
    def my_other_method(self):
        pass
    
    @staticmethod
    def my_static_method():
        pass
    
    @classmethod
    def my_class_method(cls):
        pass

# 获取MyClass对象的所有属性和方法
all_attributes = dir(MyClass)

# 遍历所有属性，标注哪些是函数哪些是变量
for attribute in all_attributes:
    # 判断属性是否是函数，并打印函数名称和类型
    if callable(getattr(MyClass, attribute)):
        print(f"{attribute} is a function")
    # 如果不是函数，则打印属性名称和类型
    else:
        print(f"{attribute} is a variable")

In [ ]:
"""
Script name: my_script.py
Author: DengJun Lei(EEEINLD)
Date created: March 28, 2023
Description: Collect the configuration information for multiple 4480 radios.
Process: 
        1.Extract file returned by 'carrierListHandler print table' command.
        2.Extract (version carrierType rfPort dl/ul Band Power)
        3.Batch extraction and output to a table. 
        4.Export origin data based on a single radio.
        5.Export summary information based on the configuration information of all radios.
"""

###################### User Define ###############################
# Files directory Location
self.dir_path = 'C:\\Users\\MSI-NB\\Downloads\\6651_config_Denmark_68_Nodes\\6651_config_Denmark_68_Nodes'
# Config Location
self.config_path = 'D:\\projects\\ericsson_flow\\configs\\4480_Configure_Statistic_Config.ecfg'
# Output Location
self.output_path = 'D:\\projects\\ericsson_flow\\output'
# Save name
self.save_name = '6651_config_Denmark_68_Nodes'

###################### Batch handle file, according to config ###############################
table = await self.batch_handle(self.dir_path, self.config_path)
table = table.drop(['search_atoms', 'chart_atoms', 'statistic_atoms'], axis=1)

###################### Export origin data ###############################
items = table['table'].values
data = pd.DataFrame()
for index, item in enumerate(items):
    try:
        tmp = pd.DataFrame(item)
    except Exception as e:
        print(str(e))
        continue
    data = data.append(tmp, ignore_index=True)
    
columns = ['SiteName', 'Bxp', 'Version']
tmpl = sorted(list(set(data.columns).difference(set(columns))))
band1 = []
band3 = []
for c in tmpl:
    if 'BandIII_' in c:
        band3.append(c)
    elif 'BandI_' in c:
        band1.append(c)
        
columns.extend(band1)
columns.extend(band3)
data = data.reindex(columns=columns)
data.to_csv(f'{self.output_path}//{self.save_name}_origin_data.csv', index=False)
await self.on_console(msg='Export Origin Data Finish!')

###################### Export summary data ###############################
data = data.fillna('')
columns = ['SiteName', 'Bxp', 'Version']
tmpl = sorted(list(set(data.columns).difference(set(columns))))
band1 = []
band3 = []
for c in tmpl:
    if 'BandIII_' in c:
        band3.append(c)
    elif 'BandI_' in c:
        band1.append(c)
        
band1.extend(band3)
columns = band1
total = len(data)
res = data.groupby(columns).size().reset_index(name='RadioAmount')
for index,item in enumerate(json.loads(res.to_json(orient='records'))):
    band1_dbm = []
    band3_dbm = []
    total_dbm = 0
    for key in item.keys():
        if ('output_power_BandIII' in key) & (item[key] != ''):
            band3_dbm.append(item[key])
        elif ('output_power_BandI' in key) & (item[key] != ''):
            band1_dbm.append(item[key])
    band1_dbm = sum(band1_dbm) if len(band1_dbm) > 0 else 0
    band3_dbm = sum(band3_dbm) if len(band3_dbm) > 0 else 0
    total_dbm = band1_dbm + band3_dbm
    res.loc[index, ['BandI_dBm', 'BandIII_dBm', 'BandI+BandIII_dBm']] = [band1_dbm,band3_dbm,total_dbm]
res['Percentage'] = res['RadioAmount'] / total
res.to_csv(f'{self.output_path}//{self.save_name}_summary.csv', index=False)
await self.on_console(msg='Export Summary Finish!')

In [ ]:
"""
Collect the configuration information for 4480 radios xslx.

Author: DengJun Lei(EEEINLD)
CreateTime: 2023.3.20
ChangeTime: 2023.3.30
Version: 1.1
"""

###################### User Define ###############################
# Files directory Location
self.xslx_path = 'D:\\projects\\ericsson_flow\\CC summary V2,0.xlsx'
# Output Location
self.output_path = 'D:\\projects\\ericsson_flow\\output'

print('Loading file...')
sheet_names = pd.read_excel(self.xslx_path, sheet_name=None).keys()
for sheet_name in sheet_names:
    print(f'Start Handle {sheet_name}...')
    data = pd.read_excel(self.xslx_path, sheet_name=sheet_name, engine='openpyxl')
    
    ###################### Export origin data ###############################
    result = []
    x= list(data['SerialNo'].values)
    sns = sorted(set(x), key=x.index)
    for sn in sns:
        radio = data.loc[(data['SerialNo'] == sn)&(data['MaxTxPower_br'].notna()), :].reset_index(drop=True)
        if len(radio) == 0:
            continue
        item = {'SerialNo': sn, 'SwPack': list(radio['SwPack'].values)[0]}
        for port,band,power in radio[['rfPortId', 'freqBand', 'MaxTxPower_br']].values:
            if band in [1,3]:
                item['Band'+str(int(band))+'_Port'+port+'_MaxTxPower_br'] = power

        band1_power = []
        band3_power = []
        for key in item.keys():
            if 'Band1' in key:
                band1_power.append(item[key])
            if 'Band3' in key:
                band3_power.append(item[key])

        item['Band1_MaxTxPower_br'] = sum(band1_power) if len(band1_power) > 0 else 0
        item['Band3_MaxTxPower_br'] = sum(band3_power) if len(band3_power) > 0 else 0
        item['Band1+Band3_MaxTxPower_br'] = item['Band1_MaxTxPower_br'] + item['Band3_MaxTxPower_br']
        result.append(item)
    data = ''
    del data
    origin_data = pd.DataFrame(result)
    origin_data.to_csv(f'{self.output_path}\\{sheet_name}_Origin_Data.csv', index=False)
    
    ###################### Export summary data ###############################
    origin_data = origin_data.fillna('')
    total = len(origin_data)
    summary = origin_data.groupby(list(origin_data.columns[2:])).size().reset_index(name='RadioAmount')
    summary['Percentage'] = summary['RadioAmount'] / total
    summary.to_csv(f'{self.output_path}\\{sheet_name}_Summary.csv', index=False)

In [ ]:
"""
Script name: my_script.py
Author: DengJun Lei(EEEINLD)
Date created: March 28, 2023
Description: Collect the configuration information for multiple 4480 radios.
Process: 
        1.Extract file returned by 'carrierListHandler print table' command.
        2.Extract (version carrierType rfPort dl/ul Band Power)
        3.Batch extraction and output to a table. 
        4.Export origin data based on a single radio.
        5.Export summary information based on the configuration information of all radios.
"""

###################### User Define ###############################
# Files directory Location
self.file_path = 'D:\\Projects\\ericsson_flow\\new_files\\ru_lock_unlock_normal1_simple.log'
# Config Location
self.config_path = 'D:\\Projects\\ericsson_flow\\new_files\\6419ConfigSimple.ecfg'

new_file_namespace = self.text_analysis.file_container.namespace+'/'+create_uuid4()
tmp_file = await TextFileModel(self.text_analysis.file_container, new_file_namespace, self.file_path, 'batch')

await self.on_console(msg='Loading config wait...')
await tmp_file.on_load_all_config('', self.config_path)

# data_tree = tmp_file.on_get_roles(_type='tree')
data_list = tmp_file.on_get_roles(_type='list')

def linear_scale(domain, range):
    def scale(x):
        return np.interp(x, domain, range)
    return scale

pixel_width = 5000
align_type = 'timestamp'
line_story_height = 20
line_chart_height = 200
interval_height = 10

line_type = 'dash'

glyph = []
# define xaxis
start_x = []
end_x = []
for node in data_list:
    if node['data'] is not None:
        if align_type == 'timestamp':
            if(node['data']['start_timestamp'] != 0) & (node['data']['end_timestamp'] != 0):
                start_x.append(node['data']['start_timestamp'])
                end_x.append(node['data']['end_timestamp'])
        else:
            start_x.append(node['data']['start_global_index'])
            end_x.append(node['data']['end_global_index'])
glyph.append({
    'type': 'XAxis',
    'pixel_width': pixel_width,
    'lower_bound': min(start_x),
    'upper_bound': max(end_x)
})

inter_global = linear_scale([min(start_x), max(end_x)], [0, pixel_width])
# define indented tree
line_stories = []
line_charts = []
current_height = 0
for node in data_list:
    # print(node['identifier'], current_height)
    if node['data'] is None:
        node['data'] = {}
        node['data']['sx'] = 0
        node['data']['sy'] = current_height
        node['data']['height'] = line_story_height
        current_height = current_height + line_story_height + interval_height
    else:
        node['data']['sy'] = current_height
        if align_type == 'timestamp':
            if (node['data']['start_timestamp'] != 0) & (node['data']['end_timestamp'] != 0):
                node['data']['sx'] = inter_global(node['data']['start_timestamp'])
                node['data']['ex'] = inter_global(node['data']['end_timestamp'])
            else:
                node['data']['sx'] = 0
                node['data']['ex'] = 0
        else:
            node['data']['sx'] = inter_global(node['data']['start_global_index'])
            node['data']['ex'] = inter_global(node['data']['end_global_index'])
            
        inter_local = linear_scale([node['data']['sx'], node['data']['ex']], [0, node['data']['ex'] - node['data']['sx']])
        if node['data']['type'] == 'chart':
            for key in node['data']['select_lines'].keys():
                for dot in node['data']['select_lines'][key]:
                    if align_type == 'timestamp':
                        dot['x'] = inter_local(inter_global(dot['timestamp']))
                    else:
                        dot['x'] = inter_local(inter_global(dot['global_index']))
            node['data']['width'] = node['data']['ex'] - node['data']['sx']
            node['data']['height'] = line_chart_height
            node['data']['line_type'] = line_type
            current_height = current_height + line_chart_height + interval_height
            line_charts.append(node)
        elif node['data']['type'] == 'search':
            for key in node['data']['res_marks'].keys():
                for dot in node['data']['res_marks'][key]:
                    if align_type == 'timestamp':
                        dot['x'] = inter_local(inter_global(dot['timestamp']))
                    else:
                        dot['x'] = inter_local(inter_global(dot['global_index']))
                        
            if (len(node['data']['res_compare_special_lines']) > 0):
                for dot in node['data']['res_compare_special_lines']:
                    if align_type == 'timestamp':
                        dot['x'] = inter_local(inter_global(dot['timestamp']))
                    else:
                        dot['x'] = inter_local(inter_global(dot['global_index']))
            node['data']['height'] = line_story_height
            node['data']['top_triangles'] = node['data']['res_marks']
            node['data']['bottom_triangles'] = node['data']['res_compare_special_lines']
            current_height = current_height + line_story_height + interval_height
            line_stories.append(node)
    tmp_file.roles.update_node(node["identifier"], data=node["data"])

data_tree = tmp_file.on_get_roles(_type='tree')
glyph.append({
    'type': 'IndentedTree',
    'data': [{'identifier': '6419','data':data_tree}]
})
glyph.append({
    'type': 'LineStory',
    'data': line_stories
})
glyph.append({
    'type': 'LineChart',
    'data': line_charts
})

await self.on_draw('', glyph)
await tmp_file.on_delete('')

In [7]:
import numpy as np

inter_global = np.interp(x=[5.2], xp=[5, 6], fp=[0, 10])
inter_global

array([2.])

In [10]:
from treelib import Node, Tree

# 创建一个示例树
tree = Tree()
tree.create_node("Node 1", "1", data={"key": "value1"})
tree.create_node("Node 2", "2", parent="1", data={"key": "value2"})

# 输出树的结构，包括节点 ID 和默认属性
node_list = tree.to_dict(sort=True, with_data=True)
# for node_dict in node_list:
#     node_id = node_dict["data"]["id"]
#     node_data = node_dict["data"]["key"]
#     node_children = node_dict.get("children", [])
#     print(f"Node {node_id}: data={node_data}, children={node_children}")

In [11]:
node_list

{'Node 1': {'children': [{'Node 2': {'data': {'key': 'value2'}}}],
  'data': {'key': 'value1'}}}